In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [6]:
from utils.data_utils import process_data_table 
raw_data = pd.read_json('data/train.json')
all_prompts = process_data_table(raw_data)

In [7]:
from utils.eval_utils import find_answer, evaluate_maths

In [8]:
all_prompts[0].get('answer')

'14.1%'

In [9]:
all_prompts[0].get('step_by_step_answers')

[206588.0, 181001.0, 25587.0, 0.14136]

In [13]:
diff_percentage_fix

np.float64(9.99)

In [ ]:
prediction = 0.1
ground_truth = 10

difference = np.abs(prediction - ground_truth)
diff_percentage_fix = difference



In [23]:
difference

np.float64(9.9)

In [4]:
test = r"""
    To calculate the percentage change in gross unrecognized tax benefits from 2011 to 2012:  

    1. **Net Change**:  
    \text{subtract}(2062, 1375)  

    2. **Percentage Change**:  
    \boxed{\text{multiply}\left(\text{divide}\left(\text{subtract}(2062, 1375), 1375\right), 100\right)}  

    (No unit normalization required, as all values are in the same currency.)
"""

In [5]:
test

'\n    To calculate the percentage change in gross unrecognized tax benefits from 2011 to 2012:  \n\n    1. **Net Change**:  \n    \\text{subtract}(2062, 1375)  \n\n    2. **Percentage Change**:  \n    \\boxed{\\text{multiply}\\left(\\text{divide}\\left(\\text{subtract}(2062, 1375), 1375\\right), 100\\right)}  \n\n    (No unit normalization required, as all values are in the same currency.)\n'

In [8]:
evaluate_maths(find_answer(test))

49.96363636363637

In [29]:
test.replace('\\', '').replace('boxed', '').replace('{', '').replace('}', '').replace('text', '').replace('left','').replace('right','')

'multiply(divide(subtract(2062, 1375), 1375), 100)'

In [30]:
((2062 - 1375) /1375) * 100

49.96363636363637

In [ ]:
multiply(divide(subtract(2062, 1375), 1375), 100)

In [24]:
diff_percentage_fix

np.float64(0.0)

In [3]:
from utils.model_utils import converse_llm, async_converse_llm
from openai import OpenAI, AsyncOpenAI
import os

from utils.prompts import SYSTEM_PROMPT_V3

In [ ]:
client = AsyncOpenAI(
    api_key=os.environ.get('DEEPSEEK_API'),
    base_url="https://api.deepseek.com",
)


import asyncio
tasks = []
task = asyncio.create_task(async_converse_llm(
    processed_data_entry=all_prompts[0],
    client=client,
    model_name='deepseek-reasoner',
    use_gold_inds=True,
    question_to_use='step_by_step_questions',
    answers_to_use='step_by_step_answers',
    sys_prompt=SYSTEM_PROMPT_V3,
))
tasks.append(task)

test = await asyncio.gather(*tasks)

In [8]:
client = OpenAI(
    api_key=os.environ.get('DEEPSEEK_API'),
    base_url="https://api.deepseek.com",
)
test = converse_llm(
    processed_data_entry=all_prompts[0],
    client=client,
    model_name='deepseek-math',
    use_gold_inds=True,
    question_to_use='step_by_step_questions',
    answers_to_use='step_by_step_answers',
    sys_prompt=SYSTEM_PROMPT_V3,
)

In [10]:
client.models.list()

SyncPage[Model](data=[Model(id='deepseek-chat', created=None, object='model', owned_by='deepseek'), Model(id='deepseek-reasoner', created=None, object='model', owned_by='deepseek')], object='list')

In [5]:
test

[{'question': 'what is the net cash from operating activities in 2009?',
  'complete_response': 'Error encountered: Connection error.',
  'model_response': '',
  'annotator_answer': 206588.0},
 {'question': 'what about in 2008?',
  'complete_response': 'To determine the net cash from operating activities in 2008 based on the provided context:  \n- The value explicitly stated for 2008 is **$206,588** (no calculations required).  \n\nFinal answer:  \n\\boxed{206588}',
  'model_response': '206588',
  'annotator_answer': 181001.0},
 {'question': 'what is the difference?',
  'complete_response': 'The difference between the net cash from operating activities in 2008 ($206,588) and 2009 ($174,247) is calculated as:\n\n\\boxed{subtract(206588, 174247)}  \n\n**Intermediate unit normalization**:  \n- 2008 value: $206,588 (no normalization required)  \n- 2009 value: $174,247 (no normalization required)',
  'model_response': 'subtract(206588,174247)',
  'annotator_answer': 25587.0},
 {'question': 